#  LE2: Datenbereinigung (Data Cleaning)

Die Bereinigung der Rohdaten ist ein wichtiger Schritt, um sicherzustellen, dass die Daten für deskriptive Analysen und die spätere Aggregation (LE3) valide und frei von Artefakten sind.

In diesem Abschnitt wird der bereits durchgeführte Schritt der Imputation fehlender Werte dokumentiert und die Daten werden systematisch auf folgende Unsauberkeiten überprüft:

1.  **Duplikate:** Vollständig doppelte Zeilen.
2.  **Datenfehler/Inkonsistenzen:** Falsche oder unrealistische Werte (z.B. unrealistische Temperaturen).
3.  **Fehlende Werte (Missing Values):** Überprüfung, ob nach der ersten Imputation noch welche existieren.
4.  **Ausreißer (Outliers):** Identifizierung von Extremwerten, deren Einfluss in LE3 mit dem robusten Schätzer, wie zum Beispiel Trimmed Mean, gemindert wird.

Jetzt laden wir uns erstmal den bearbeiteten Datensatz wie in LE1 herunter.

In [2]:
import kagglehub
import pandas as pd

# --- LE1: Importieren und Laden ---
# 1. Datensatz herunterladen und Pfad abrufen. Dies behebt den FileNotFoundError.
path = kagglehub.dataset_download(handle="nicholasjhana/energy-consumption-generation-prices-and-weather")

print(f"Path to dataset files: {path}")

# Dateipfade setzen
path_energy = path + "/energy_dataset.csv"
path_weather = path + "/weather_features.csv"

# 2. Datasets laden
energy_df = pd.read_csv(path_energy)
weather_df = pd.read_csv(path_weather)
print("Datasets erfolgreich geladen.")


# --- LE1: Datentypen konvertieren (Zeitreihen) ---
energy_df['time'] = pd.to_datetime(energy_df['time'], utc=True)
energy_df = energy_df.set_index('time')

weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'], utc=True)
weather_df = weather_df.set_index('dt_iso')
print("Zeitstempel als Index gesetzt.")


# --- LE2: Fehlende Werte (Erste Imputation durch den Kollegen) ---
# Ersetzt fehlende 'total load actual' Werte durch den 'total load forecast'
energy_df['total load actual'].fillna(energy_df['total load forecast'], inplace=True)
print("Erste Imputation für 'total load actual' durchgeführt.")

Path to dataset files: /Users/ilyas/.cache/kagglehub/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather/versions/1
Datasets erfolgreich geladen.
Zeitstempel als Index gesetzt.
Erste Imputation für 'total load actual' durchgeführt.


/var/folders/2g/2kljx72s71qfpqfsl03p810w0000gn/T/ipykernel_90741/2889404782.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  energy_df['total load actual'].fillna(energy_df['total load forecast'], inplace=True)


### 3.1. Duplikate: Prüfung und Entfernung (Check 1 von 4)

Wir prüfen beide DataFrames (`energy_df` und `weather_df`) auf vollständig duplizierte Zeilen. Im Weather DataFrame wird speziell auf Duplikate von Zeitstempel und Stadtname geprüft, um sicherzustellen, dass keine doppelten Messungen vorliegen.

In [3]:
# --- Duplikate prüfen und behandeln (LE2) ---

# Energy DataFrame
# Duplikate: Prüft, ob alle Spalten (inkl. Index) in einer Zeile identisch sind.
num_duplicates_energy = energy_df.duplicated().sum()
print(f"\nAnzahl der Duplikate im Energy DataFrame: {num_duplicates_energy}")

if num_duplicates_energy > 0:
    energy_df.drop_duplicates(inplace=True)
    print("Duplikate im Energy DataFrame wurden entfernt.")
else:
    print("Keine Duplikate im Energy DataFrame gefunden.")

# --- Duplikate im Weather DataFrame prüfen und behandeln ---
# Duplikate: Prüft, ob alle Spalten (inkl. Index/Zeitstempel) in einer Zeile identisch sind.
num_duplicates_weather = weather_df.duplicated().sum()

print(f"Anzahl der Duplikate (gleiche Zeile inkl. Index) im Weather DataFrame: {num_duplicates_weather}")

if num_duplicates_weather > 0:
    weather_df.drop_duplicates(inplace=True)
    print("Duplikate im Weather DataFrame wurden entfernt.")
else:
    print("Keine Duplikate im Weather DataFrame gefunden.")


Anzahl der Duplikate im Energy DataFrame: 0
Keine Duplikate im Energy DataFrame gefunden.
Anzahl der Duplikate (gleiche Zeile inkl. Index) im Weather DataFrame: 8622
Duplikate im Weather DataFrame wurden entfernt.


Wir prüfen, ob die Zeilen zufällig die selbe Temperatur hat, oder ob sie wirklich 1 zu 1 kopiert sind. Dafür gehen wir auf den Timestemp ein und analysieren diese für jede Stadt.

In [7]:
# --- Prüfung auf Duplikate in den Schlüsselspalten (dt_iso, city_name) ---

# Der Zeitstempel (dt_iso) ist bereits der Index.
# Wir prüfen, ob der Index zusammen mit dem city_name noch Duplikate bildet.
key_cols = ['city_name', weather_df.index.name]

# Temporäres Zurücksetzen des Index, um ihn als Spalte in 'subset' zu verwenden
weather_temp_df = weather_df.reset_index()

# Zählen der Duplikate nur anhand der Schlüsselspalten (Zeit + Stadt)
partial_duplicates_count = weather_temp_df.duplicated(subset=key_cols, keep=False).sum()

print(f"\nAnzahl der Zeilen, die den gleichen Zeitstempel und Stadt aufweisen: {partial_duplicates_count}")


Anzahl der Zeilen, die den gleichen Zeitstempel und Stadt aufweisen: 5795


In [8]:
# --- 1. Konfliktspalten definieren ---
key_cols = ['city_name', 'dt_iso'] # Wir müssen den Index wieder als Spalte definieren

# Temporäres Zurücksetzen des Index, um ihn als Spalte in 'subset' zu verwenden
weather_temp_df = weather_df.reset_index()


# --- 2. Duplikate identifizieren ---
# Identifiziere alle Zeilen, die mehrfach mit derselben Zeit/Stadt vorkommen.
conflicting_rows_mask = weather_temp_df.duplicated(subset=key_cols, keep=False)


# --- 3. Konfliktzeilen anzeigen und sortieren ---
# Zeige die Zeilen, die in den Schlüsselspalten doppelt vorkommen.
conflicting_df = weather_temp_df[conflicting_rows_mask].sort_values(by=['dt_iso', 'city_name'])


print(f"Anzahl der Konfliktzeilen (müssen manuell geprüft werden): {conflicting_df.shape[0]}")
print("\nErste 10 konfliktäre Beobachtungen (gleiche Zeit/Stadt, unterschiedliche Werte):")
print(conflicting_df.head(10))

Anzahl der Konfliktzeilen (müssen manuell geprüft werden): 5795

Erste 10 konfliktäre Beobachtungen (gleiche Zeit/Stadt, unterschiedliche Werte):
                          dt_iso   city_name    temp  temp_min  temp_max  \
137639 2015-03-04 05:00:00+00:00     Seville  288.47    285.15    291.15   
137640 2015-03-04 05:00:00+00:00     Seville  288.47    285.15    291.15   
104631 2015-03-20 20:00:00+00:00   Barcelona  286.81    282.59    291.48   
104632 2015-03-20 20:00:00+00:00   Barcelona  286.81    282.59    291.48   
106225 2015-06-08 13:00:00+00:00   Barcelona  299.47    291.48    304.82   
106226 2015-06-08 13:00:00+00:00   Barcelona  299.47    291.48    304.82   
106227 2015-06-08 14:00:00+00:00   Barcelona  297.59    292.04    300.37   
106228 2015-06-08 14:00:00+00:00   Barcelona  297.59    292.04    300.37   
106229 2015-06-08 15:00:00+00:00   Barcelona  297.03    292.04    300.15   
106230 2015-06-08 15:00:00+00:00   Barcelona  297.03    292.04    300.15   

        pressure 

In [10]:
weather_df.iloc[[104631, 104632]]

,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
dt_iso,,,,,,,,,,,,,,,,
2015-03-20 20:00:00+00:00,Barcelona,286.81,282.59,291.48,1013,76,12,50,0.3,0.0,0.0,40,500,rain,light rain,10n
2015-03-20 20:00:00+00:00,Barcelona,286.81,282.59,291.48,1013,76,12,50,0.3,0.0,0.0,40,301,drizzle,drizzle,09n


Wir sehen, dass die Zeilen identisch aussehen, ausser die Spalten weather_id, weather_description und weather_icon.

### 3.2. Ergebnisse der Duplikate-Prüfung und Vorgehen bei fehlenden Werten (Check 2 von 4)

#### Ergebnisse Duplikate
Die Prüfung auf vollständige Duplikate ergab folgende Ergebnisse:

* **Energy DataFrame (`energy_df`):** Es wurden **keine** Duplikate gefunden.
* **Weather DataFrame (`weather_df`):** Es wurden **8622** vollständig identische, redundante Zeilen gefunden. Diese Redundanz wurde behoben, indem die Duplikate entfernt wurden (`drop_duplicates()`). Dies stellt die Konsistenz und Unabhängigkeit der stündlichen Wetterbeobachtungen sicher.

#### Nächstes Vorgehen: Fehlende Werte
Nach der erfolgreichen Behandlung der Duplikate konzentrieren wir uns auf die **verbleibenden fehlenden Werte (Missing Values)**.

Obwohl der wichtigste Missing Value im `energy_df` bereits durch **Imputation** ersetzt wurde, ist eine Überprüfung aller verbleibenden Spalten notwendig, um sicherzustellen, dass keine Datenlücken die nachfolgenden Transformations- oder Verknüpfungsschritte (LE3, LE4) behindern.

In [4]:
# --- Prüfung auf verbleibende Missing Values (LE2) ---

print("\n--- Fehlende Werte im Energy DataFrame ---")
# Zeigt die Summe der fehlenden Werte pro Spalte
missing_energy = energy_df.isnull().sum()
print(missing_energy[missing_energy > 0]) # Nur Spalten mit fehlenden Werten anzeigen

print("\n--- Fehlende Werte im Weather DataFrame ---")
missing_weather = weather_df.isnull().sum()
print(missing_weather[missing_weather > 0]) # Nur Spalten mit fehlenden Werten anzeigen


--- Fehlende Werte im Energy DataFrame ---
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                                  18
ge


Die Prüfung auf fehlende Werte ergab, dass das `weather_df` nun vollständig ist. Im `energy_df` wurden jedoch zwei Kategorien von Missing Values identifiziert:

1.  **Massive Fehlwerte (35.064 Zeilen):** Die Spalten 'generation hydro pumped storage aggregated' und 'forecast wind offshore eday ahead' sind fast komplett leer und werden daher aus dem Datensatz entfernt.
2.  **Geringfügige Fehlwerte (17-19 Zeilen):** In den restlichen 'generation'-Spalten fehlen nur einzelne Werte. Diese werden durch den **Median** der jeweiligen Spalte imputiert, um die Datensätze vollständig zu machen und die Robustheit gegenüber möglichen Ausreißern zu gewährleisten.

In [5]:
# --- 1. Entfernung nicht-brauchbarer Spalten ---
# Spalten mit > 99% fehlenden Werten entfernen
cols_to_drop = ['generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead']
energy_df.drop(columns=cols_to_drop, inplace=True)
print(f"Spalten {cols_to_drop} aufgrund zu vieler fehlender Werte entfernt.")

# --- 2. Imputation der geringfügigen Fehlwerte ---
# Wir nutzen den Median für die Imputation (guter und robuster Ansatz für numerische Daten)
for col in energy_df.columns:
    if energy_df[col].isnull().sum() > 0:
        median_value = energy_df[col].median()
        energy_df[col].fillna(median_value, inplace=True)

# Überprüfung des Ergebnisses
remaining_na = energy_df.isnull().sum().sum()
print(f"Gesamtzahl der verbleibenden fehlenden Werte im Energy DataFrame: {remaining_na}")

Spalten ['generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead'] aufgrund zu vieler fehlender Werte entfernt.
Gesamtzahl der verbleibenden fehlenden Werte im Energy DataFrame: 0


/var/folders/2g/2kljx72s71qfpqfsl03p810w0000gn/T/ipykernel_90741/1922116625.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  energy_df[col].fillna(median_value, inplace=True)
/var/folders/2g/2kljx72s71qfpqfsl03p810w0000gn/T/ipykernel_90741/1922116625.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

Zur Behandlung fehlender Werte (NA) in der Spalte wurde für unsere Arbeit dann doch eine Imputationsstrategie gewählt, indem die NA-Werte durch die korrespondierenden Werte als Forecast ersetzt wurden. Diese Vorgehensweise demonstriert die Anwendung problemgerechter Imputationsstrategien, die über einfache Substitutionen wie den Median (den wir hier demonstriert haben) hinausgehen und die zeitliche Abhängigkeit der Daten berücksichtigen.

### 3.4. Ausreißer (Outliers) und Datenfehler (Check 3 & 4 von 4)

#### Ausreißerstrategie
Obwohl in der Explorativen Datenanalyse (EDA) in den Zeitreihen- und Verteilungsplots (besonders im 'total load actual') potenzielle Ausreißer identifiziert wurden, werden diese **nicht direkt entfernt**.

Stattdessen wenden wir in der nachfolgenden Transformationsphase (LE3) einen **robusten Schätzer** an, wie zum Beispiel die **Trimmed Mean Aggregation**. Diese Methode mildert den verzerrenden Einfluss extremer Ausreißer auf den Mittelwert, ohne die Datenpunkte physisch aus dem Datensatz zu löschen.

#### Datenfehler/Inkonsistenzen
Nach der erfolgreichen Bereinigung der Duplikate und fehlenden Werte und der Verwendung robuster Schätzer für Ausreißer, wird angenommen, dass der Datensatz nun ausreichend **konsistent** für die weiteren Transformationsschritte (LE3) ist.

---

## ✅ Abschluss LE2: Bereinigen

Die DataFrames sind nun vollständig bereinigt und strukturiert für die weiteren Schritte der Transformation und Verknüpfung.

### 4. Speichern der bereinigten Daten (Pipeline-Übergabe)

Um die Reproduzierbarkeit (LE6) und die Effizienz der Pipeline (LE5) zu gewährleisten, werden die bereinigten DataFrames (`energy_df` und `weather_df`) im effizienten Parquet-Format gespeichert. Sie dienen als Input für die nächste Phase (LE3: Transformation/Aggregation).

In [6]:
# Speichern des bereinigten Energy DataFrame
energy_df.to_parquet('energy_cleaned.parquet')

# Speichern des bereinigten Weather DataFrame
weather_df.to_parquet('weather_cleaned.parquet')

print("Bereinigte DataFrames erfolgreich als 'energy_cleaned.parquet' und 'weather_cleaned.parquet' gespeichert.")

Bereinigte DataFrames erfolgreich als 'energy_cleaned.parquet' und 'weather_cleaned.parquet' gespeichert.
